In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Dataframe setting

In [ ]:
df = pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')

# Dataframe check

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

# Target variable "deposit"

In [ ]:
sns.countplot(x = "deposit", data = df, label = "Count")
plt.show()

In [ ]:
# Change to binary
df['deposit'] = df['deposit'].astype('category')
df['deposit'] = df['deposit'].cat.codes

In [ ]:
# Check the binary number : 0 : no, 1 : yes
sns.countplot(x = "deposit", data = df, label = "Count")
plt.show()

# Feature Analysis

In [ ]:
# Check the existence of null number
print(df.isnull().sum())

### [Categorical variables]

"job"

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x = "job", data = df, label = "Count")
plt.show()

In [ ]:
# Factrial plot

sns.factorplot(x = "job", y = "deposit", data = df, kind = "bar", size = 12)
plt.show()

Obs:
* Some categories have same factorial level. It indicates that there are possibilities to integrate categories.
* For exemple, "blue-collar" and "entrepreneur", "service" and "housemaid", "technician" and "self-employed".
* However, we should consider the meanings of categories.
* Maybe we can integrate "service" and "housemaid" as "housemaid" is possibly one of service.
* Some "technician" could be "self-employed", then, we can think abount this integration.
* On the ohter hand, it is difficult to consider that "blue-collar" and "entrepreneur" are similar categories. In my opinion, it is not good to integrate them.
* If you want to integrate the categories and make the new variable, the following is the processing.

In [ ]:
def fac_job(row):
  if row['job'] == "housemaid":
    val = "service"
  elif row['job'] == "technician":
    val = "self-employed" 
  else :
    val = row['job']
  return val

In [ ]:
df['job.rev'] = df.apply(fac_job, axis=1)

"marital"

In [ ]:
sns.countplot(x = "marital", data=df, hue = "deposit")
plt.show()

In [ ]:
sns.factorplot(x = "marital", y = "deposit", data = df, kind = "bar", size = 6)
plt.show()

Obs:
* Obseving the figures, the categories are reasonable and could react separately to the target "deposit".

"education"

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x = "education", data = df, hue = "deposit")
plt.show()

In [ ]:
sns.factorplot(x = "education", y = "deposit", data = df, kind = "bar", size = 6)
plt.show()

Obs:
* If you want, we could integrate "primary" and "secondary" as they have similar factorial level. But I think the categories are reasonable.
* We could make the class by number such as 1: primary, 2: secondary, 3: tertiary, however, it is difficult how to classify the "unknown". Then it might be treated as the categories separately.

"default"

In [ ]:
sns.countplot(x ="default", data = df, hue = "deposit")
plt.show()

In [ ]:
sns.factorplot(x = "default", y = "deposit", data = df, kind = "bar", size = 6)
plt.show()

Obs:
* Obseving the figures, "yes" and "no" have different factors. Then I analyze that "default" could improve to fit models.

"housing"

In [ ]:
sns.countplot(x = "housing", data = df, hue = "deposit")
plt.show()

In [ ]:
sns.factorplot(x = "housing", y = "deposit", data = df, kind = "bar", size = 6)
plt.show()

Obs:
* Similarly, "yes" and "no" have different factors. Then I analyze that "default" could improve to fit models.

In [ ]:
sns.countplot(x = "loan", data = df,label = "Count")
plt.show()

In [ ]:
sns.factorplot(x = "loan", y = "deposit", data = df, kind = "bar", size = 6)
plt.show()

Obs:
* Similarly, "yes" and "no" have different factors. Then I analyze that "default" could improve to fit models.

In [ ]:
sns.countplot(x = "contact", data = df, hue = "deposit")
plt.show()

In [ ]:
sns.factorplot(x = "contact", y = "deposit", data = df, kind = "bar", size = 6)
plt.show()

Obs:
* "cellular" and "telephone" have similar factorial level and "unknown"'s level is very different from theirs. Then one possibility is to integrate them and make a category such as "telecomunication" for example. It makes the variable "contact" be binary.
* However, I respect the original categories for now.

"month"

In [ ]:
sns.countplot(x = "month", data = df, hue= "deposit")

In [ ]:
sns.factorplot(x = "month", y = "deposit", data = df, kind = "bar", size = 10)
plt.show()

"poutcome"

In [ ]:
sns.countplot(x = "poutcome", data = df, hue= "deposit")
plt.show()

In [ ]:
sns.factorplot(x = "poutcome", y = "deposit", data = df, kind = "bar", size = 6)
plt.show()

Obs:
* There are different factorial levels. Then I would like to respect the original categories.

### [Numerical variables]

In [ ]:
plt.figure(figsize=(15,5))
sns.heatmap(df.corr(), annot=True, fmt = ".2f", cmap = "coolwarm")

In [ ]:
plt.figure(figsize=(15,5))
sns.pairplot(df.corr())

Obs :
* We can recognize in the heatmap that there is a considebly high correlation between "deposit" and "duration", then "duration" should be importatn variable. However they are not correlated really linearly seeing the pairplot. 
* "campaign", "pdays" and "previous" correlate with deposit even though it is not high.
* The correlation between "pdays" and "previous" is able to be observed, then we should be careful when we choose the variables to modeling as they could conflict some times and reduce the performance.
* There are no normal distribution. The algorithm that accepts non-normality should work better.

In [ ]:
# Example to see the relation between "depoist" and "duration" by Box-plot
sns.boxplot(x = "deposit", y = "duration", data = df)
plt.show()

In [ ]:
# Example to see the relation between "depoist" and "duration" by Violin-plot
sns.violinplot(x = "deposit", y = "duration", data = df)
plt.show()

# PCA (Principal Components Analysis)

As we could see that there is the significant correlation between "pdays" and "previous". As the number of variable is not large, there is doubt about the necessity of PCA. However, I would like to apply PCA for these two variable as a option to solve the problem of modeling with variables correlated each other.

In [ ]:
from sklearn.preprocessing import StandardScaler

features = ['pdays', 'previous']

x_pca = df.loc[:, features].values
x_pca = StandardScaler().fit_transform(x_pca)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

principalComponents = pca.fit_transform(x_pca)
principalDF = pd.DataFrame(data = principalComponents, columns=['PC1', 'PC2'])

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# Concatenation of dataframes
df_all = pd.concat([principalDF, df], axis=1)

In [ ]:
plt.figure(figsize=(15,5))
sns.heatmap(df_all.corr(), annot=True, fmt = ".2f", cmap = "coolwarm")

Obs: As we can see, PC1 (the first principal component) has better correlation with "deposit" than "pdays" and "previous". "pdays" and "previous" could be replaced with PC1 to test if the performance of algorithm will be better or not. Furthermore, PC1 represent 75.4% of these variables, then, it seems that PCA could extract main effect to deposit efficiently.

# Data processing

In [ ]:
df_all.info()

* Firstly, I select every variables to fit models but without the principal components (PC1 and PC2).
* It is ideal to select specific variables before fitting models, however, if the number of variables is not huge, sometimes it is better to see how algorithms fit model with every variables and how much is performance. Some algorithms make the feature selection by regulations as well.

In [ ]:
#### Without PCs

variables_selected = [
    'age',
    'job',
    'marital',
    'education',
    'default',
    'balance',
    'housing',
    'loan',
    'contact',
    'day',
    'month',
    'duration',
    'campaign',
    'pdays',
    'previous',
    'poutcome',
    'deposit', # target variable
]

In [ ]:
df_final = df_all[variables_selected]

In [ ]:
df_final.head()

In [ ]:
# One-hot (creating dummies for categorical variables)
df_final = pd.get_dummies(df_final, columns=['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome'])

In [ ]:
df_final.head()

# Modeling 1 (without Principal Component)

In [ ]:
y = df_final.deposit
X = df_final.drop('deposit', axis=1)

In [ ]:
y.shape, X.shape

In [ ]:
# Normalization of data
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

In [ ]:
df_X_norm = pd.DataFrame(X_norm)

In [ ]:
df_X_norm.head()

In [ ]:
# Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Metrics
from sklearn.model_selection import  GridSearchCV
from sklearn import model_selection
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score

In [ ]:
seed = 123
k = 10

kfolds = model_selection.KFold(n_splits=k, shuffle = True, random_state=seed)
#kfolds = model_selection.StratifiedKFold(n_splits=k, random_state=seed)
# In this case the proportion of responce "yes" and "no" is balanced well. If not, it is recommended to use "StratifiedKFold()"

In [ ]:
### KNN

KNN = KNeighborsClassifier()

params = {
    'n_neighbors' : list(range(2,40)),
    'weights' : ['distance'],
    'algorithm' : ['kd_tree']
}

grid_knn = GridSearchCV(KNN, param_grid=params, cv = kfolds)
grid_knn.fit(X_norm, y)

In [ ]:
print(grid_knn.best_score_)
print(grid_knn.best_params_)

In [ ]:
#### Decision Tree

DT = DecisionTreeClassifier()

params = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : list(range(2, 15)),
}

grid_dt = GridSearchCV(DT, param_grid=params, cv=kfolds)
grid_dt.fit(X_norm, y)

In [ ]:
print(grid_dt.best_score_)
print(grid_dt.best_params_)

In [ ]:
### Logistic regression

LR = LogisticRegression()

params = {
    'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 100),
    'solver' : ['liblinear']
}

grid_lr = GridSearchCV(LR, param_grid=params, cv = kfolds)
grid_lr.fit(X_norm, y)

In [ ]:
print(grid_lr.best_score_)
print(grid_lr.best_params_)

In [ ]:
### Random Forest

RF = RandomForestClassifier()

params = {
    "n_estimators": np.array([50, 100, 200, 300]), 
    "max_depth": [3, 6, 9, 12, 15, 18, 21],
}

grid_rf = GridSearchCV(RF, param_grid=params, cv = kfolds)
grid_rf.fit(X_norm, y)

In [ ]:
print(grid_rf.best_score_)
print(grid_rf.best_params_)

In [ ]:
### XG Boost

xgb = XGBClassifier(verbosity = 0) # verbosity = 0 : to hide warnings

params = {
    'max_depth': [6, 9, 11, 13, 16, 19, 22], 
    'n_estimators': [100],
    'learning_rate': [0.01, 0.05, 0.1]
}

grid_xgb = GridSearchCV(xgb, param_grid=params, cv = kfolds)
grid_xgb.fit(X_norm, y)


In [ ]:
print(grid_xgb.best_score_)
print(grid_xgb.best_params_)

In [ ]:
### SVM

svm = SVC()

params = {
    "C": [0.001, 0.01, 0.1, 1.],
    "kernel": ["linear", "poly", "rbf"],
    "gamma": ["scale", "auto"], 
}

grid_svm = GridSearchCV(svm, param_grid=params, cv = kfolds)
grid_svm.fit(X_norm, y)


In [ ]:
print(grid_svm.best_score_)
print(grid_svm.best_params_)

# Modeling with Principal Component "PC1"

In [ ]:
#### With PC1 and "job.rev"

variables_selected = [
    'age',
    'job.rev', # categories reduced
    'marital',
    'education',
    'default',
    'balance',
    'housing',
    'loan',
    'contact',
    'day',
    'month',
    'duration',
    'campaign',
    #'pdays', # excluded
    #'previous', # excluded
    'PC1', # included
    'poutcome',
    'deposit', # target variable
]

In [ ]:
df_final = df_all[variables_selected]

In [ ]:
df_final.head()

In [ ]:
# One-hot (creating dummies for categorical variables)
df_final = pd.get_dummies(df_final, columns=['job.rev', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome'])

In [ ]:
df_final.head()

In [ ]:
y = df_final.deposit
X = df_final.drop('deposit', axis=1)

In [ ]:
# Normalization of data

scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

In [ ]:
df_X_norm = pd.DataFrame(X_norm)

In [ ]:
df_X_norm.head()

In [ ]:
### KNN

KNN = KNeighborsClassifier()

params = {
    'n_neighbors' : list(range(2,40)),
    'weights' : ['distance'],
    'algorithm' : ['kd_tree']
}

grid_knn = GridSearchCV(KNN, param_grid=params, cv = kfolds)
grid_knn.fit(X_norm, y)

In [ ]:
print(grid_knn.best_score_)
print(grid_knn.best_params_)

In [ ]:
#### Decision Tree

DT = DecisionTreeClassifier()

params = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : list(range(2, 15)),
}

grid_dt = GridSearchCV(DT, param_grid=params, cv=kfolds)
grid_dt.fit(X_norm, y)

In [ ]:
print(grid_dt.best_score_)
print(grid_dt.best_params_)

In [ ]:
### Logistic regression

LR = LogisticRegression()

params = {
    'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 100),
    'solver' : ['liblinear']
}

grid_lr = GridSearchCV(LR, param_grid=params, cv = kfolds)
grid_lr.fit(X_norm, y)

In [ ]:
print(grid_lr.best_score_)
print(grid_lr.best_params_)

In [ ]:
### Random Forest

RF = RandomForestClassifier()

params = {
    "n_estimators": np.array([50, 100, 200, 300]), 
    "max_depth": [3, 6, 9, 12, 15, 18, 21],
}

grid_rf = GridSearchCV(RF, param_grid=params, cv = kfolds)
grid_rf.fit(X_norm, y)

In [ ]:
print(grid_rf.best_score_)
print(grid_rf.best_params_)

In [ ]:
### XG Boost

xgb = XGBClassifier(verbosity = 0) # verbosity = 0 : to hide warnings

params = {
    'max_depth': [6, 9, 11, 13, 16, 19, 22], 
    'n_estimators': [100],
    'learning_rate': [0.01, 0.05, 0.1]
}

grid_xgb = GridSearchCV(xgb, param_grid=params, cv = kfolds)
grid_xgb.fit(X_norm, y)


In [ ]:
print(grid_xgb.best_score_)
print(grid_xgb.best_params_)

In [ ]:
### SVM

svm = SVC()

params = {
    "C": [0.001, 0.01, 0.1, 1.],
    "kernel": ["linear", "poly", "rbf"], 
    "gamma": ["scale", "auto"],
}

grid_svm = GridSearchCV(svm, param_grid=params, cv = kfolds)
grid_svm.fit(X_norm, y)


In [ ]:
print(grid_svm.best_score_)
print(grid_svm.best_params_)